# RNN for Classification

In [1]:
import os
import sys
import torch

import pandas as pd

import torch.nn as nn

notebook_dir = os.getcwd()

sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
predictions_df = DataProcessing.load_single_synthetic_data(notebook_dir, predictions=True, batch_idx=2)
predictions_df

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv


,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"Based on my analysis as a financial expert, I forecast that the stock price at Tesla will potentially decrease in Q3 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Goldman Sachs speculates that the net profit at Microsoft will likely increase in the following fiscal year.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,2
2,"JP Morgan Chase predicts that on January 1, 2025, the revenue of Amazon may rise significantly.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,3
3,"According to a report by BlackRock, the operating cash flow at Google's parent company, Alphabet, would fall in the second quarter of 2025.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,4
4,"In Q2 of 2025, a financial research advisor envisions that the research and development expenses at Johnson & Johnson have some probability to remain stable.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,5
5,"The trading volume of ExxonMobil should stay the same in the third quarter of 2025, according to a college student studying finance.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,6
6,A financial advisor predicts that the net profit at Procter & Gamble potentially decrease in 2026 Q2.,1,finance,mistral-small-3.1,NAVI_GATOR,0,1
7,"On 08/21/2024, a senior financial analyst speculates the revenue at Chevron will likely increase.",1,finance,mistral-small-3.1,NAVI_GATOR,0,2
8,"A financial expert predicts on 21 August 2024, the Johnson & Johnson stock price may rise.",1,finance,mistral-small-3.1,NAVI_GATOR,0,3
9,"According to a financial top executive, the operating income at Boeing would fall in 2025 Q3.",1,finance,mistral-small-3.1,NAVI_GATOR,0,4


In [4]:
observations_df = DataProcessing.load_single_synthetic_data(notebook_dir, predictions=False)
observations_df

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_7-observation/batch_7-from_df.csv


,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase observed that the operating income at Microsoft had increased significantly in September 2027.,0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,1
1,"On November 10, 2024, to November 10, 2025, Citigroup monitored the research and development expenses at Amazon and saw it changed.",0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,2
2,"BlackRock noted on February 2, 2026, the S&P 500 index at Goldman Sachs fell.",0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,3
3,"According to Visa, the net profit at Mastercard rose in the fourth quarter of 2028.",0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,4
4,"In January 2029, Wells Fargo envisioned that the dividend yield at Coca-Cola decreased.",0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,5
5,"The stock price of Nike increased in October 2025, according to a financial analyst at Barclays.",0,finance,llama-3.3-70b-versatile,GROQ_CLOUD,0,6
6,Morgan Stanley observed that the revenue at FUBU (his parents clothing line) had increased for Q3 2028.,0,finance,llama-3.1-8b-instant,GROQ_CLOUD,0,1
7,"On 2024-08-21, Goldman Sachs monitored the operating cash flow at UF's school of Engineering and saw it decreased.",0,finance,llama-3.1-8b-instant,GROQ_CLOUD,0,2
8,Morgan Stanley noted that the ETFs in his portfolio exponentially grew from 21 Aug 2024 to 2024/08/21.,0,finance,llama-3.1-8b-instant,GROQ_CLOUD,0,3
9,"According to Chase Bank, the returns at emerging market equities fell in May 2035.",0,finance,llama-3.1-8b-instant,GROQ_CLOUD,0,4


## Data Processing

In [5]:
sentence_labels = []
prediction_sentence_label = int(predictions_df['Sentence Label'].unique())
observation_sentence_label = int(observations_df['Sentence Label'].unique())
sentence_labels.append(prediction_sentence_label)
sentence_labels.append(observation_sentence_label)
print(sentence_labels)

[1, 0]


/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_22079/1923887029.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction_sentence_label = int(predictions_df['Sentence Label'].unique())
/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_22079/1923887029.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  observation_sentence_label = int(observations_df['Sentence Label'].unique())


In [6]:
mappings = {}
mappings[prediction_sentence_label] = list(predictions_df['Base Sentence'].values)
mappings[observation_sentence_label] = list(observations_df['Base Sentence'].values)
mappings

{1: ['Based on my analysis as a financial expert, I forecast that the stock price at Tesla will potentially decrease in Q3 of 2024.',
  'On August 21, 2024, Goldman Sachs speculates that the net profit at Microsoft will likely increase in the following fiscal year.',
  'JP Morgan Chase predicts that on January 1, 2025, the revenue of Amazon may rise significantly.',
  "According to a report by BlackRock, the operating cash flow at Google's parent company, Alphabet, would fall in the second quarter of 2025.",
  'In Q2 of 2025, a financial research advisor envisions that the research and development expenses at Johnson & Johnson have some probability to remain stable.',
  'The trading volume of ExxonMobil should stay the same in the third quarter of 2025, according to a college student studying finance.',
  'A financial advisor predicts that the net profit at Procter & Gamble potentially decrease in 2026 Q2.',
  'On 08/21/2024, a senior financial analyst speculates the revenue at Chevron

In [7]:
import spacy

# Lazy loading - only load models when first used
_embedding_models = {}

def get_embedding_model(model_name):
    if model_name not in _embedding_models:
        model_map = {
            'spacy_medium': 'en_core_web_md',
            'spacy_large': 'en_core_web_lg',
        }
        _embedding_models[model_name] = spacy.load(model_map[model_name])
    return _embedding_models[model_name]

def embed_sentence(sentence, embedding_model_name='spacy_medium'):
    model = get_embedding_model(embedding_model_name)
    doc = model(sentence)  # Call model directly, not model.nlp
    return doc.vector  # Returns sentence embedding as numpy array

In [8]:
input_embedding = embed_sentence(mappings[1][0], 'spacy_large')
torch_embedding = torch.tensor(input_embedding)
print(f"torch_embedding size: {torch_embedding.size()}")

torch_embedding size: torch.Size([300])


## Network

In [9]:
class RNN(nn.Module):
    """Specify size for each layer of the RNN"""
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.input_hidden_to_hidden = nn.Linear(in_features=input_size + hidden_size, out_features=hidden_size)
        self.input_hidden_to_output = nn.Linear(in_features=input_size + hidden_size, out_features=output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, hidden_tensor):
        """Perform computation for RNN with the parameter to each attribute is proper size as initiated"""
        input_to_hidden = torch.cat((input_tensor, hidden_tensor), 1)
        # print(f"input_to_hidden size: {input_to_hidden.size()}")

        hidden = self.input_hidden_to_hidden(input_to_hidden)
        output = self.input_hidden_to_output(input_to_hidden)
        output = self.softmax(output)
        return hidden, output
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [10]:
data_length = len(predictions_df) + len(observations_df)
hidden_size = 128
rnn = RNN(300, hidden_size, len(sentence_labels))
rnn

RNN(
  (input_hidden_to_hidden): Linear(in_features=428, out_features=128, bias=True)
  (input_hidden_to_output): Linear(in_features=428, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [11]:


resized_torch_embedding =  torch_embedding.unsqueeze(0)
print(f"resized_torch_embedding size: {resized_torch_embedding.size()}")

hidden_embedding = rnn.init_hidden()
print(f"hidden_embedding size: {hidden_embedding.size()}")

output, next_hidden = rnn(resized_torch_embedding, hidden_embedding)
print(output.size())
print(next_hidden.size())

resized_torch_embedding size: torch.Size([1, 300])
hidden_embedding size: torch.Size([1, 128])
torch.Size([1, 128])
torch.Size([1, 2])


In [12]:
def get_model_label(output):
    # print(output)
    idx = torch.argmax(output).item()
    # print(idx)
    
    # Ensure idx is within valid range
    if idx < 0 or idx >= len(sentence_labels):
        raise ValueError(f"Model predicted index {idx} but only {len(sentence_labels)} labels exist")
    
    return sentence_labels[idx]

In [13]:
# get_model_label(output)

In [14]:
import random
def random_training_example(category_lines, all_categories):
    
    def random_choice(a):
        random_idx = random.randint(0, len(a) - 1)
        return a[random_idx]
    
    category = random_choice(all_categories)
    line = random_choice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = embed_sentence(line, 'spacy_large')
    return category, line, category_tensor, line_tensor


In [15]:
criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

# Fixed train function - no iteration needed for sentence-level
def train(line_tensor, category_tensor):
    hidden = rnn.init_hidden()
    
    # Convert to tensor and add batch dimension
    line_tensor = torch.tensor(line_tensor).unsqueeze(0)
    
    # Single forward pass for whole sentence
    output, hidden = rnn(line_tensor, hidden)
    
    loss = criterion(output, category_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return output, loss.item()

# Training loop
current_loss = 0
all_losses = []
plot_steps, print_steps = 1000, 5000
n_iters = 10000

for i in range(n_iters):
    category, line, category_tensor, line_tensor = random_training_example(mappings, sentence_labels)
    output, loss = train(line_tensor, category_tensor)
    current_loss += loss
    
    if (i+1) % plot_steps == 0:
        all_losses.append(current_loss / plot_steps)
        current_loss = 0
    
    if (i+1) % print_steps == 0:
        guess = get_model_label(output)  # Fixed: was sentence_labels(output)
        correct = "CORRECT" if guess == category else f"WRONG ({category})"
        print(f"{i+1} {(i+1)/n_iters*100:.1f}% {loss:.4f} {line} / {guess} {correct}")

# Evaluation function
def evaluate(sentence):
    with torch.no_grad():
        hidden = rnn.init_hidden()
        line_tensor = torch.tensor(embed_sentence(sentence, 'spacy_large')).unsqueeze(0)
        output, hidden = rnn(line_tensor, hidden)
        prediction = get_model_label(output)
        return prediction

5000 50.0% -99.3328 According to policy advisor David Lee, the voter turnout at local elections rose on 21/08/2024. / 0 CORRECT
10000 100.0% -193.2162 According to economist Dr. Sofia Lopez, the unemployment rate in the technology sector would fall in Q2 2024. / 1 CORRECT


In [16]:
test_predictions_df = DataProcessing.load_single_synthetic_data(notebook_dir, predictions=True, batch_idx=3)
test_observations_df = DataProcessing.load_single_synthetic_data(notebook_dir, predictions=False, batch_idx=9)
test_sentences_df = DataProcessing.concat_dfs([test_predictions_df, test_observations_df])

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_3-prediction/batch_3-from_df.csv
/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_9-observation/batch_9-from_df.csv


In [17]:
print("\n--- Testing ---")

results = []
test_sentences = DataProcessing.df_to_list(test_sentences_df, 'Base Sentence')
for sentence in test_sentences:
    pred = evaluate(sentence)
    result = {
        'Sentence': sentence,
        'Prediction Label': pred
    }
    results.append(result)
        

results       


--- Testing ---


[{'Sentence': 'Emily Chen, a financial analyst, forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'Prediction Label': 1},
 {'Sentence': 'On August 21, 2024, JPMorgan Chase speculates the operating income at Microsoft will likely increase.',
  'Prediction Label': 1},
 {'Sentence': 'Bank of America predicts on 2024-08-21, the research and development expenses at Alphabet may rise.',
  'Prediction Label': 1},
 {'Sentence': 'According to Citigroup, the revenue at Facebook would fall in 2025 Q2.',
  'Prediction Label': 1},
 {'Sentence': 'In 21 August 2024, Wells Fargo envisions that the gross profit at Intel has some probability to remain stable.',
  'Prediction Label': 1},
 {'Sentence': 'The stock price at Visa should stay same in Q3 of 2027, according to Mastercard.',
  'Prediction Label': 1},
 {'Sentence': 'JPMorgan forecasts that the revenue at Amazon potentially decrease in Q3 of 2027.',
  'Prediction Label': 1},
 {'Sentence': 'On 2024-08-21, Citigroup spe

In [18]:
import pandas as pd
test_df = pd.DataFrame(results)
test_df

,Sentence,Prediction Label
0,"Emily Chen, a financial analyst, forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.",1
1,"On August 21, 2024, JPMorgan Chase speculates the operating income at Microsoft will likely increase.",1
2,"Bank of America predicts on 2024-08-21, the research and development expenses at Alphabet may rise.",1
3,"According to Citigroup, the revenue at Facebook would fall in 2025 Q2.",1
4,"In 21 August 2024, Wells Fargo envisions that the gross profit at Intel has some probability to remain stable.",1
5,"The stock price at Visa should stay same in Q3 of 2027, according to Mastercard.",1
6,JPMorgan forecasts that the revenue at Amazon potentially decrease in Q3 of 2027.,1
7,"On 2024-08-21, Citigroup speculates the net profit at Microsoft will likely increase.",1
8,"BlackRock predicts on 21 August 2024, the operating income at Johnson & Johnson may rise.",0
9,"According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2025.",1
